In [1]:
import numpy as np
import pandas as pd
import torch
import plotly.express as px
import plotly.graph_objects as go
import json
import os
import imageio 
from plotly.subplots import make_subplots
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.transforms import Compose, Resize
import torch.optim as optim
import torch.nn as nn
from torchinfo import summary
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
# from pytorch_model_summary import summary
torch.random.manual_seed(42)
from mlcm import mlcm
import ipdb


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.get_device_properties(device = 'cuda:0')

cuda:0


_CudaDeviceProperties(name='NVIDIA GeForce GTX 1050 Ti', major=6, minor=1, total_memory=4096MB, multi_processor_count=6)

In [3]:
 # Reading the image and the captions dataframe and the labels dataframe which has the super categories.
image_df = pd.read_csv('F://coco/captions/final_captions.csv',)
labels_df = pd.read_csv('F://coco/captions/labels_information_full.csv')

# Also we need to read the multilabel dataframe which has the multilabel infomration about the images.
multilabel_df = pd.read_csv('F://coco/captions/multilabel_labels.csv').fillna(0)


In [4]:
image_label_df = image_df[['image_id','file_name']].merge(multilabel_df, on='image_id',validate='1:1')
image_label_df

,image_id,file_name,food,animal,furniture,electronic,kitchen,vehicle,person,outdoor,accessory,sports,appliance,indoor
0,57870,COCO_train2014_000000057870.jpg,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,384029,COCO_train2014_000000384029.jpg,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,222016,COCO_train2014_000000222016.jpg,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,520950,COCO_train2014_000000520950.jpg,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,69675,COCO_train2014_000000069675.jpg,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66809,360271,COCO_train2014_000000360271.jpg,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
66810,471345,COCO_train2014_000000471345.jpg,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66811,444010,COCO_train2014_000000444010.jpg,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
66812,565004,COCO_train2014_000000565004.jpg,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [5]:
single_channel_list = ['COCO_train2014_000000061048.jpg',
'COCO_train2014_000000434765.jpg',
'COCO_train2014_000000053756.jpg',
'COCO_train2014_000000470933.jpg',
'COCO_train2014_000000571415.jpg',
'COCO_train2014_000000492325.jpg',
'COCO_train2014_000000155083.jpg',
'COCO_train2014_000000421613.jpg',
'COCO_train2014_000000431115.jpg',
'COCO_train2014_000000173610.jpg',
'COCO_train2014_000000336668.jpg',
'COCO_train2014_000000316867.jpg',
'COCO_train2014_000000269858.jpg',
'COCO_train2014_000000250239.jpg',
'COCO_train2014_000000123539.jpg',
'COCO_train2014_000000140092.jpg',
'COCO_train2014_000000343009.jpg',
'COCO_train2014_000000003293.jpg',
'COCO_train2014_000000578250.jpg',
'COCO_train2014_000000518025.jpg',
'COCO_train2014_000000008794.jpg',
'COCO_train2014_000000400107.jpg',
'COCO_train2014_000000394547.jpg',
'COCO_train2014_000000389984.jpg',
'COCO_train2014_000000225717.jpg',
'COCO_train2014_000000032405.jpg',
'COCO_train2014_000000084582.jpg',
'COCO_train2014_000000549879.jpg',
'COCO_train2014_000000358281.jpg',
'COCO_train2014_000000457741.jpg',
'COCO_train2014_000000204792.jpg',
'COCO_train2014_000000124694.jpg',
'COCO_train2014_000000390663.jpg',
'COCO_train2014_000000179405.jpg',
'COCO_train2014_000000443909.jpg',
'COCO_train2014_000000268036.jpg',
'COCO_train2014_000000217341.jpg',
'COCO_train2014_000000363331.jpg',
'COCO_train2014_000000134071.jpg',
'COCO_train2014_000000505962.jpg',
'COCO_train2014_000000347111.jpg',
'COCO_train2014_000000484742.jpg',
'COCO_train2014_000000064270.jpg',
'COCO_train2014_000000012345.jpg',
'COCO_train2014_000000226585.jpg',
'COCO_train2014_000000575029.jpg',
'COCO_train2014_000000564314.jpg',
'COCO_train2014_000000011801.jpg',
'COCO_train2014_000000033127.jpg',
'COCO_train2014_000000312288.jpg',
'COCO_train2014_000000126531.jpg',
'COCO_train2014_000000140627.jpg',
'COCO_train2014_000000131366.jpg',
'COCO_train2014_000000406011.jpg',
'COCO_train2014_000000369966.jpg',
'COCO_train2014_000000081003.jpg',
'COCO_train2014_000000010125.jpg',
'COCO_train2014_000000006432.jpg',
'COCO_train2014_000000384693.jpg',
'COCO_train2014_000000470442.jpg',
'COCO_train2014_000000280731.jpg',
'COCO_train2014_000000113929.jpg',
'COCO_train2014_000000416869.jpg',
'COCO_train2014_000000066642.jpg',
'COCO_train2014_000000233263.jpg',
'COCO_train2014_000000025404.jpg',
'COCO_train2014_000000156878.jpg',
'COCO_train2014_000000166522.jpg',
'COCO_train2014_000000060060.jpg',
'COCO_train2014_000000445845.jpg',
'COCO_train2014_000000205486.jpg',
'COCO_train2014_000000577207.jpg',
'COCO_train2014_000000005294.jpg',
'COCO_train2014_000000186888.jpg',
'COCO_train2014_000000503640.jpg',
'COCO_train2014_000000000086.jpg',
'COCO_train2014_000000087509.jpg',
'COCO_train2014_000000571503.jpg',
'COCO_train2014_000000000821.jpg',
'COCO_train2014_000000579138.jpg',
'COCO_train2014_000000134918.jpg',
'COCO_train2014_000000259284.jpg',
'COCO_train2014_000000257178.jpg',
'COCO_train2014_000000221691.jpg',
'COCO_train2014_000000077709.jpg',
'COCO_train2014_000000263002.jpg',
'COCO_train2014_000000341892.jpg',
'COCO_train2014_000000349069.jpg',
'COCO_train2014_000000563376.jpg',
'COCO_train2014_000000220770.jpg',
'COCO_train2014_000000208206.jpg',
'COCO_train2014_000000027412.jpg',
'COCO_train2014_000000434837.jpg',
'COCO_train2014_000000080906.jpg',
'COCO_train2014_000000150354.jpg',
'COCO_train2014_000000107450.jpg',
'COCO_train2014_000000577265.jpg',
'COCO_train2014_000000416372.jpg',
'COCO_train2014_000000377837.jpg',
'COCO_train2014_000000579239.jpg',
'COCO_train2014_000000540378.jpg',
'COCO_train2014_000000525513.jpg',
'COCO_train2014_000000353952.jpg',
'COCO_train2014_000000006379.jpg',
'COCO_train2014_000000381270.jpg',
'COCO_train2014_000000520479.jpg',
'COCO_train2014_000000563447.jpg',
'COCO_train2014_000000085407.jpg',
'COCO_train2014_000000210175.jpg',
'COCO_train2014_000000397575.jpg',
'COCO_train2014_000000058517.jpg',
'COCO_train2014_000000384907.jpg',
'COCO_train2014_000000509358.jpg',
'COCO_train2014_000000264165.jpg',
'COCO_train2014_000000072098.jpg',
'COCO_train2014_000000155954.jpg',
'COCO_train2014_000000270925.jpg',
'COCO_train2014_000000104124.jpg',
'COCO_train2014_000000095753.jpg',
'COCO_train2014_000000210847.jpg',
'COCO_train2014_000000507794.jpg',
'COCO_train2014_000000561842.jpg',
'COCO_train2014_000000249835.jpg',
'COCO_train2014_000000361516.jpg',
'COCO_train2014_000000451074.jpg',
'COCO_train2014_000000480482.jpg',
'COCO_train2014_000000220898.jpg',
'COCO_train2014_000000260962.jpg',
'COCO_train2014_000000576700.jpg',
'COCO_train2014_000000296884.jpg',
'COCO_train2014_000000342921.jpg',
'COCO_train2014_000000384910.jpg',
'COCO_train2014_000000040428.jpg',
'COCO_train2014_000000145288.jpg',
'COCO_train2014_000000321897.jpg',
'COCO_train2014_000000449901.jpg',
'COCO_train2014_000000107962.jpg',
'COCO_train2014_000000001350.jpg',
'COCO_train2014_000000249711.jpg',
'COCO_train2014_000000140623.jpg',
'COCO_train2014_000000211867.jpg',
'COCO_train2014_000000496444.jpg',
'COCO_train2014_000000287422.jpg',
'COCO_train2014_000000118895.jpg',
'COCO_train2014_000000075052.jpg',
'COCO_train2014_000000436984.jpg',
'COCO_train2014_000000555583.jpg',
'COCO_train2014_000000029275.jpg',
'COCO_train2014_000000176397.jpg',
'COCO_train2014_000000034861.jpg',
'COCO_train2014_000000517899.jpg',]


In [6]:
indexes = []
for file_name in single_channel_list:
    index = image_label_df[image_label_df['file_name']==file_name].index.values[0]
    indexes.append(index)
    # print(index)
    # image_label_df.drop(, inplace=True)

In [7]:
image_label_df

,image_id,file_name,food,animal,furniture,electronic,kitchen,vehicle,person,outdoor,accessory,sports,appliance,indoor
0,57870,COCO_train2014_000000057870.jpg,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,384029,COCO_train2014_000000384029.jpg,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,222016,COCO_train2014_000000222016.jpg,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,520950,COCO_train2014_000000520950.jpg,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,69675,COCO_train2014_000000069675.jpg,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66809,360271,COCO_train2014_000000360271.jpg,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
66810,471345,COCO_train2014_000000471345.jpg,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66811,444010,COCO_train2014_000000444010.jpg,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
66812,565004,COCO_train2014_000000565004.jpg,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
image_label_df = image_label_df.drop(indexes, axis = 0)
image_label_df = image_label_df.reset_index(drop=True)
image_label_df

,image_id,file_name,food,animal,furniture,electronic,kitchen,vehicle,person,outdoor,accessory,sports,appliance,indoor
0,57870,COCO_train2014_000000057870.jpg,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,384029,COCO_train2014_000000384029.jpg,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,222016,COCO_train2014_000000222016.jpg,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,520950,COCO_train2014_000000520950.jpg,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,69675,COCO_train2014_000000069675.jpg,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66658,360271,COCO_train2014_000000360271.jpg,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
66659,471345,COCO_train2014_000000471345.jpg,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66660,444010,COCO_train2014_000000444010.jpg,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
66661,565004,COCO_train2014_000000565004.jpg,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


#### Now we we have modified the data to suit our needs. Next we will start to focus on creating the dataset out of the data and the model.


### Creating the Dataset class

In [9]:
class CocoDataset(Dataset):
    def __init__(self,image_label_df, val_stride = 10, is_val_set_bool = False, test_data_set =False, test_stride = 5):
        
        self.transfrom = Resize((224,224))
        self.is_val_set_bool = is_val_set_bool
        self.val_stride = val_stride
        self.image_label_df = image_label_df.copy()
        self.test_data_set = test_data_set
        self.test_stride = test_stride

        if self.test_data_set: #If we need only a small subset of the data to work with
            self.image_label_df = self.image_label_df[::test_stride].reset_index(drop = True)

        elif self.is_val_set_bool: # If we need only the validation data then return the validation data which is a subset of total data
            assert self.val_stride > 0
            self.image_label_df = self.image_label_df[::val_stride]

        elif self.val_stride > 0:  # Else if val_stride is greater than zero then return the remaining dataframe after removing 10% of data
            self.image_label_df = self.image_label_df.drop(index = list(range(0,len(self.image_label_df),self.val_stride)))
            
        else: # else train on the full dataset
            self.image_label_df = self.image_label_df
        
    def __len__(self):
        """ This method calculates the length of your data."""
        return len(self.image_label_df)

    # Now we will introduce function which gives us an image and its corresponding labels by id
    def get_image_by_id(self, img_id = None):
        """ This function returns an image by its id and the corresponding multiple labesl in a present/no_present binary format"""
        

        folder_path = 'F://coco/train2014/train2014/'

        if img_id == None:
            raise ValueError('Must provide IMAGE ID')

        else:
            row = self.image_label_df[self.image_label_df['image_id']==img_id]
            file_name = row['file_name'].values[0]

            
            label_array = torch.zeros(len(multilabel_df.columns[1:]),2)  # create an array of zeros which is a (num_classes, 2) array

            # Lets get the multilabel array for a particular image_id from multilabel_df
            multilabel_array = row.values[0][2:]

            # Now lets populate the label_array with values from the multilabel array
            label_array[range(len(multilabel_array)), multilabel_array] = 1

            # Get the image-data from file_name
            image_array = torchvision.io.read_image(folder_path + file_name)
            image_array = self.transfrom(image_array).to(torch.float32)
        
            return image_array, label_array


    # Now we would write the code for returning the image from index and its corresponding labels. Which would be used by the train/val dataloaders
    def __getitem__(self, ndx):

        """ This function takes in an index and returns the image and the labels of the image at that index"""
        folder_path = 'F://coco/train2014/train2014/'
        
        row = self.image_label_df.iloc[ndx]

        # Now get the image_id and the file_name of the image
        image_id = row['image_id']
        file_name = row['file_name']

        # print(image_id, file_name)
        # Now get the multilabel in the form of a numpy array
        # This array would be of the shape (num_classes, 2). Which means that if a class is present or not present and we will keep 1 at that
        # Input probabilities for each class wo
        label_array = torch.zeros(len(self.image_label_df.columns[2:]),2)  # create an array of zeros which is a (num_classes, 2) array


        # Now we will use the multiple_labels array to populate the categories which are present in the picture.
        # For each category we will populate 1 at the 0th position if it is not present or 1 at the 1th position if it is present
        # This way we can check for the prescence or absence of multiple categories which makes it a multilabel classification problem

        

        # Lets get the multilabel array for a particular image_id
        multilabel_array = row.values[2:].astype(np.float32)

        # Now lets populate the label_array with values from the multilabel array
        label_array[range(len(multilabel_array)), multilabel_array] = 1

        # Now get the image_data from storage
        image_array = torchvision.io.read_image(folder_path + file_name)
        image_array = self.transfrom(image_array).to(torch.float32)
        return (image_array, label_array, image_id)


            

In [11]:
# coco = CocoDataset(image_label_df=image_label_df, test_data_set=True, test_stride=5)
# img, labels, id = coco[3]
# print(img.shape, labels.shape)
# px.imshow(img.permute(1,2,0).numpy())

### Creating the train and val dataloaders.

In [10]:
train_coco = CocoDataset(image_label_df=image_label_df, val_stride=5)
val_coco = CocoDataset(image_label_df=image_label_df,is_val_set_bool=True)

# train_dataloader = DataLoader(dataset=train_coco, batch_size=128, pin_memory=True, drop_last=True)
# val_dataloader = DataLoader(dataset=val_coco, batch_size=128,pin_memory=True, drop_last = True)

# test_coco = CocoDataset(image_label_df=image_label_df, test_data_set=True, test_stride=50)

# test_coco_dataloader = DataLoader(dataset=test_coco, batch_size=128, pin_memory=True, drop_last=True)

In [13]:
len(train_coco)

53330

In [14]:
for image, label, id in train_dataloader:
    print(image.shape, label.shape )
    break

torch.Size([128, 3, 128, 128]) torch.Size([128, 12, 2])


### Creating the Model for predicting the classes
* Sequential ConvNet :
    1. First we will create a simple Convnet model which would predict the mutiple classes and train the network.
    2. This would have 5 Convolution layers and fully connected layers.
    3. It would also have 12 heads which would predict the presence or absence of the 12 classes in the coco dataset.
    4. Each head would have a binary crossentropy loss as the metric and we would average the losses of all the heads to get the final loss.

In [29]:
class SeqConvnet(nn.Module):
    def __init__(self, n_chan = 64):
        super().__init__()

        # First we will put channels in the n_chan variable
        self.n_chan = n_chan
        # Now we would define the first convlution layer which would take in a (B,N,H,w) tensor of images as input and perform convolution
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels=n_chan, kernel_size=3,padding='same') # in_shape = (32,3,400,400) | out_shape = (32,64,400,400)
        # A maxpool and a relu 
        self.max_pool1 = nn.MaxPool2d(2)
        # self.tanh1 = nn.Tanh()
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels = n_chan, out_channels = n_chan//2, kernel_size= 3, padding = 'same') # shape = (32,64,200,200) | out_shape = (32,32,200,200)
        # A maxpool and a relu 
        self.max_pool2 = nn.MaxPool2d(2)
        # self.tanh2 = nn.Tanh()
        self.relu2 = nn.ReLU()
        
        self.conv3 = nn.Conv2d(in_channels = n_chan//2, out_channels=n_chan//4, kernel_size=3, padding='same') # shape = (32,32,200,200) | out_shape = (32,16,100,100)
        # A maxpool and a relu 
        self.max_pool3 = nn.MaxPool2d(2)
        # self.tanh3 = nn.Tanh()
        self.relu3 = nn.ReLU()

        self.conv4 = nn.Conv2d(in_channels = n_chan//4, out_channels=n_chan//8, kernel_size=3, padding='same') # shape = (32,16,50,50) | out_shape = (32,8,50,50)
        self.max_pool4 = nn.MaxPool2d(2) # shape = (32,8,50,50) | out_shape = (32,8,25,25)
        # self.tanh4 = nn.Tanh()
        self.relu4 = nn.ReLU()

        # Now we will define the fully connected layers
        self.fc1 = nn.Linear((n_chan//8)*8*8, 4096) # in_shape = (32,160000) | out_shape = (32,1024)
        self.batchnorm1 = nn.BatchNorm1d(4096)
        # self.tanh5 = nn.Tanh()
        self.relu5 = nn.ReLU()
        self.fc2 = nn.Linear(4096,1024) # in_shape = (32,1024) | out_shape = (32,256)
        self.batchnorm2 = nn.BatchNorm1d(1024)
        # self.tanh6 = nn.Tanh()
        self.relu6 = nn.ReLU()
        self.fc3 = nn.Linear(1024, 128) # in_shape = (32,256) | out_shape = (32,128)
        self.batchnorm3 = nn.BatchNorm1d(128)
        # self.tanh7 = nn.Tanh()
        self.relu7 = nn.ReLU()
        self.head1 = nn.Linear(128,2)
        self.sig1 = nn.Sigmoid()
        self.head2 = nn.Linear(128,2)
        self.sig2 = nn.Sigmoid()
        self.head3 = nn.Linear(128,2)
        self.sig3 = nn.Sigmoid()
        self.head4 = nn.Linear(128,2)
        self.sig4 = nn.Sigmoid()
        self.head5 = nn.Linear(128,2)
        self.sig5 = nn.Sigmoid()
        self.head6 = nn.Linear(128,2)
        self.sig6 = nn.Sigmoid()
        self.head7 = nn.Linear(128,2)
        self.sig7 = nn.Sigmoid()
        self.head8 = nn.Linear(128,2)
        self.sig8 = nn.Sigmoid()
        self.head9 = nn.Linear(128,2)
        self.sig9 = nn.Sigmoid()
        self.head10 = nn.Linear(128,2)
        self.sig10 = nn.Sigmoid()
        self.head11 = nn.Linear(128,2)
        self.sig11 = nn.Sigmoid()
        self.head12 = nn.Linear(128,2)
        self.sig12 = nn.Sigmoid()
        # self.head_dict = {}
        # for i in range(12):
        #     name = f"head{i}"
        #     self.head_dict[name] = nn.Linear(128,2)   # # in_shape = (32,128) | out_shape = (32,2)
    
    def forward(self, x):
        # Define the computations of the convolution block
        out = self.max_pool1(self.conv1(x))
        # print(out.shape)
        # print(out)
        out = self.relu1(out)
        out = self.max_pool2(self.conv2(out))
        out = self.relu2(out)
        out = self.max_pool3(self.conv3(out))
        out = self.relu3(out)
        out = self.max_pool4(self.conv4(out))
        out = self.relu4(out)

        # Now define the computations of the fully connected layers.
        # print(out.shape)
        out = out.view(128,-1)
        # print(out.shape)
        # assert out.shape == (32, 160000)
        out = self.relu5(self.batchnorm1(self.fc1(out)))
        out = self.relu6(self.batchnorm2(self.fc2(out)))
        out = self.relu7(self.batchnorm3(self.fc3(out)))

        # Now we will define the computations of the 12 heads and heads and put all the outputs into one tensor and return that tensor.
        # The out tensor would of the shape (12,32,2).
        self.out_head1 = self.sig1(self.head1(out))  # 'food'
        self.out_head2 = self.sig2(self.head2(out)) # animal
        self.out_head3 = self.sig3(self.head3(out)) # furniture
        self.out_head4 = self.sig4(self.head4(out)) # electronic
        self.out_head5 = self.sig5(self.head5(out)) # kitchen
        self.out_head6 = self.sig6(self.head6(out)) # vehicle
        self.out_head7 = self.sig7(self.head7(out)) # person
        self.out_head8 = self.sig8(self.head8(out)) # outdoor
        self.out_head9 = self.sig9(self.head9(out)) # accessory
        self.out_head10 = self.sig10(self.head10(out)) # sports
        self.out_head11 = self.sig11(self.head11(out)) # appliance
        self.out_head12 = self.sig12(self.head12(out)) # indoor
        out_list = [self.out_head1,self.out_head2,self.out_head3,self.out_head4,self.out_head5,self.out_head6,self.out_head7,self.out_head8,
                    self.out_head9,self.out_head10,self.out_head11,self.out_head12]
        # out_dict = {}
        # for i,key in enumerate(self.head_dict.keys()):
        #     name = f"out_head{i}"
        #     layer = self.head_dict[key]
        #     out_dict[name] = layer(out)
        # print(out_dict)
        # print(out_dict['out_head0'].shape)
        # out_tensor = torch.stack(list(out_dict.values()), dim=1)
        # print(f"the final output after all the heads is {out_tensor.shape}")

        out_tensor = torch.stack(out_list, dim = 1)
        # print(f"the shape of the final output tensor is {out_tensor.shape}")

        return out_tensor




        




In [11]:
exp_train_data, exp_train_label = [],[]

for i in range(128):
    img, labels,_ = train_coco[i]
    exp_train_data.append(img)
    exp_train_label.append(labels)
# exp_train_data = torch.tensor(exp_train_data)
# exp_train_label = torch.tensor(exp_train_label)
# print(exp_train_data.shape)
# print(exp_train_label.shape)
# print(len(exp_train_data), len(exp_train_label))
exp_train_data = torch.stack(exp_train_data)
exp_train_label = torch.stack(exp_train_label)
exp_train_data.shape

torch.Size([128, 3, 224, 224])

In [17]:
# exp_train_label[0].shape

In [18]:
# conv = SeqConvnet()
# o = conv(exp_train_data)
# print(o[0])

In [19]:
# conv = SeqConvnet()
# o = conv(exp_train_data)
# print(o[0])
# c = torch.nn.BCELoss()
# c(o, exp_train_label)

### Now we have the made the model. Next we need to test our model with some dummy data so that we can see that we are getting the correct outputs

So we can see that the model outputs the desired shapes of the ouputs which is 12 tensors of shape (32,2) i.e one tensor for each head of the class.
```
    out_tensor = (batch_size)*(Number_of_classes)*(present/not_present)
```
This woud be compared to our labels via the binary_crossentopy metric and we would then calculate the loss of individual heads and then we would average the individual losses and find our the model loss. Then we will backpropagate the loss.

In [16]:
def validation_loop(model, val_dataloader, device, loss_fn):
    print("Validation is progressing")
    val_loss_list = []
    model.eval()
    with torch.no_grad():
        for val_img, val_label, _ in tqdm(val_dataloader, nrows = 1, disable=True):
            # transfer the images and labels to the GPU
            val_img = val_img.to(device)
            val_label = val_label.to(device)

            # Now run it through the model and get the val_logits
            val_logits = model(val_img)

            val_loss = loss_fn(val_logits, val_label)
            out_val_loss = val_loss.clone().to(torch.device('cpu'))
            val_loss_list.append(out_val_loss.item())
            # val_loss += out_val_loss.item()
            # val_i +=1
    return val_loss_list

In [24]:
# Now we will begin to write the training loop that will ingest the data and output the result.
def training_loop(epochs, conv_model,train_dataloader,device,optimizer= None, val_dataloader=None, loss_fn = None):
    
    for epoch in trange(epochs):
        train_losses = []
        # print("Training is Progressing")
        for img, label,_ in tqdm(train_dataloader, nrows=1):
            img = img.to(device)
            label = label.to(device)

            # Now put the training data into the model to get the logits
            pred = conv_model(img)

            # Now we will zero out the optimizer
            optimizer.zero_grad()
            
            # calculate the loss
            loss = loss_fn(pred, label)

            out_loss = loss.clone().to(device=torch.device('cpu'))

            # backpropagate
            loss.backward()
            # Next we will update the parameters
            optimizer.step()
            train_losses.append(out_loss.item())
        
        # val_loss_list = validation_loop(model = conv_model, val_dataloader=val_dataloader, device = device, loss_fn= loss_fn)
        print(f"Epoch: {epoch} | Train_loss: {torch.tensor(train_losses).mean()}") # | Val_loss : {val_loss_list.mean()}")
            

    return 



1. Min loss reached with SGD = 0.441 with lr = 0.1     # We used the tanh non-linearity without batchnorm
2. Min loss reached with Adam = 0.47 with a lr = 0.1   # we used the tanh non-linearity without batchnorm
3. Min loss reached with Adam = 0.442 with a lr = 0.1, weight_decay = 0.01   # we used the relu non-linearity without batchnorm\
previously we were not able to achieve good losses because we were not calculating the output tensor correctly, becuase it is a multilabel classification problem which predicts the whether a class is present or not present. The mistake that we were doing is that we were not calculating the probabilities of each head and just inputting the logits to the binary crossentopy loss. Instead what we should have done is that we should calculate the probabilities of prescence or absence of each class and then feed the output into the Binary crossentropy loss.
4. Min loss that we achieved with Adam = 0.72 with a lr = 0.001  # we used relu without batchnorm(this was after we did the above fix)


In [31]:
summary(seq_convnet)

Layer (type:depth-idx)                   Param #
SeqConvnet                               --
├─Conv2d: 1-1                            1,792
├─MaxPool2d: 1-2                         --
├─ReLU: 1-3                              --
├─Conv2d: 1-4                            18,464
├─MaxPool2d: 1-5                         --
├─ReLU: 1-6                              --
├─Conv2d: 1-7                            4,624
├─MaxPool2d: 1-8                         --
├─ReLU: 1-9                              --
├─Conv2d: 1-10                           1,160
├─MaxPool2d: 1-11                        --
├─ReLU: 1-12                             --
├─Linear: 1-13                           2,101,248
├─BatchNorm1d: 1-14                      8,192
├─ReLU: 1-15                             --
├─Linear: 1-16                           4,195,328
├─BatchNorm1d: 1-17                      2,048
├─ReLU: 1-18                             --
├─Linear: 1-19                           131,200
├─BatchNorm1d: 1-20              

In [32]:
# First we will instantiate the model
seq_convnet = SeqConvnet()
bce_loss = torch.nn.BCELoss()
seq_convnet = seq_convnet.to(device=device)
optimizer_adam = optim.Adam(seq_convnet.parameters(), lr = 0.001)
schedular = optim.lr_scheduler.ExponentialLR(optimizer=optimizer_adam, gamma=0.9, verbose=True)

Adjusting learning rate of group 0 to 1.0000e-03.


In [33]:
training_loop(10,device = device, conv_model=seq_convnet, train_dataloader=test_coco_dataloader, optimizer=optimizer_adam, loss_fn=bce_loss)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0 | Train_loss: 0.6493959426879883


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Train_loss: 0.5123721361160278


...

Epoch: 7 | Train_loss: 0.24504156410694122


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 8 | Train_loss: 0.2105756551027298


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 9 | Train_loss: 0.18622803688049316


The model that we trained above was trained with the intention of overfitting a small subset of data. Now we will check the metrics of the model.

In [28]:
summary(seq_convnet)

Layer (type:depth-idx)                   Param #
SeqConvnet                               --
├─Conv2d: 1-1                            1,792
├─MaxPool2d: 1-2                         --
├─ReLU: 1-3                              --
├─Conv2d: 1-4                            18,464
├─MaxPool2d: 1-5                         --
├─ReLU: 1-6                              --
├─Conv2d: 1-7                            4,624
├─MaxPool2d: 1-8                         --
├─ReLU: 1-9                              --
├─Conv2d: 1-10                           1,160
├─MaxPool2d: 1-11                        --
├─ReLU: 1-12                             --
├─Linear: 1-13                           262,656
├─BatchNorm1d: 1-14                      1,024
├─ReLU: 1-15                             --
├─Linear: 1-16                           131,328
├─BatchNorm1d: 1-17                      512
├─ReLU: 1-18                             --
├─Linear: 1-19                           32,896
├─BatchNorm1d: 1-20                     

In [47]:
loss_list

[0.4105244576931,
 0.40146052837371826,
 0.32220539450645447,
 0.41178712248802185,
 0.4429898262023926,
 0.44441360235214233,
 0.37629207968711853,
 0.40180474519729614,
 0.420118510723114,
 0.43926286697387695,
 0.42059749364852905,
 0.34016185998916626,
 0.3703218400478363,
 0.3791232705116272,
 0.4567175805568695,
 0.4625483751296997,
 0.38726913928985596,
 0.36139461398124695,
 0.3509107828140259,
 0.4264286458492279,
 0.4681588411331177,
 0.43750593066215515,
 0.3688054084777832,
 0.4028228223323822,
 0.4283648133277893,
 0.47029441595077515,
 0.42021259665489197,
 0.33833691477775574,
 0.32548588514328003,
 0.37759456038475037,
 0.4459021985530853,
 0.4668058753013611,
 0.4027005136013031,
 0.40450185537338257,
 0.3765893876552582,
 0.45167598128318787,
 0.4379539489746094,
 0.36664098501205444,
 0.3533148467540741,
 0.37280043959617615,
 0.4211289882659912,
 0.4412575364112854,
 0.34760135412216187,
 0.3974948823451996,
 0.38323959708213806,
 0.4345957040786743,
 0.447017431259

In [48]:
# Lets save the model
torch.save(seq_convnet.state_dict(), 'F://coco/saved_models/seq_convnet_batch_norm_full.pt')

In [49]:
seq_convnet = SeqConvnet()
seq_convnet.load_state_dict(torch.load('F://coco/saved_models/seq_convnet_batch_norm_full.pt'))
seq_convnet.to(device=device)

SeqConvnet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu1): ReLU()
  (conv2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu2): ReLU()
  (conv3): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (max_pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu3): ReLU()
  (conv4): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (max_pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu4): ReLU()
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu5): ReLU()
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (batchnorm2): Bat

Now we will se how the model performs on when classifying the images 

In [ ]:
# # First we will get the logits and then what we will do is that we will get the probabilities associated with each class
# with torch.no_grad():
#     for data, labels, _ in tqdm(val_dataloader, nrows = 1, disable=True):
#         data = data.to(device)
#         labels = labels.to(device)
#         val_logits = seq_convnet(data)
#         loss = F.binary_cross_entropy_with_logits(val_logits, labels)
#         # print(loss.item())
#         break

Now lets create a multilabel confusion matrix to see how our model performs. We have used a custom library for multi-label confusion matrix called as mlcm. This was specifically designed for multilabel classes.

In [50]:
# confusion_mat = torch.zeros(12,12)
def make_confusion_mat(logits_tensor, labels_tensor):
    logits_tensor_argmax = torch.argmax(logits_tensor, dim = 2)
    labels_tensor_argmax = torch.argmax(labels_tensor, dim = 2)
    batch_conf_mat = mlcm.cm(labels_tensor_argmax.numpy(), logits_tensor_argmax.numpy(), print_note=False)
    
    return batch_conf_mat

# px.imshow(confusion_mat)

Now we will add each batch confusion matrix to another matrix and then we would visualize the confusion matrix for the whole training set.

In [51]:

conf_mat = torch.zeros((13,13))
seq_convnet.eval()
seq_convnet = seq_convnet.to(device=device)
with torch.no_grad():
    
    for data, label, _ in val_dataloader:
        data = data.to(device = device)
        
        # label = label.to(device = device)
        logits = seq_convnet(data)
        label = label.to(device= torch.device('cpu'))
        logits = logits.to(device= torch.device('cpu'))
        batch_conf_mat = make_confusion_mat(logits_tensor= logits, labels_tensor=label)
        conf_mat += batch_conf_mat[0]


In [52]:
conf_mat

tensor([[2.3600e+02, 2.5000e+01, 7.0000e+01, 0.0000e+00, 2.0000e+00, 1.0000e+00,
         4.2000e+01, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         5.6800e+02],
        [5.0000e+00, 5.6000e+02, 2.2000e+01, 0.0000e+00, 1.0000e+00, 4.0000e+00,
         7.6000e+01, 0.0000e+00, 0.0000e+00, 6.0000e+00, 0.0000e+00, 0.0000e+00,
         6.5600e+02],
        [1.9000e+01, 5.6000e+01, 2.9400e+02, 0.0000e+00, 2.0000e+00, 8.0000e+00,
         1.0000e+02, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         1.1820e+03],
        [6.0000e+00, 1.6000e+01, 3.5000e+01, 0.0000e+00, 2.0000e+00, 3.0000e+00,
         5.7000e+01, 0.0000e+00, 0.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00,
         6.2200e+02],
        [1.7000e+01, 2.4000e+01, 6.0000e+01, 0.0000e+00, 1.1000e+01, 3.0000e+00,
         8.8000e+01, 0.0000e+00, 0.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00,
         9.8300e+02],
        [1.0000e+01, 1.6700e+02, 1.7000e+01, 0.0000e+00, 0.0000e+00, 1.1000e+02,

In [53]:
img = px.imshow(conf_mat, color_continuous_scale='tempo')
img.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0,1, 2, 3, 4, 5,6, 7,8, 9,10, 11],
        ticktext =['food', 'animal', 'furniture', 'electronic', 'kitchen', 'vehicle',
       'person', 'outdoor', 'accessory', 'sports', 'appliance', 'indoor']
    ),
    yaxis = dict
    (
        tickmode = 'array',
        tickvals = [0,1, 2, 3, 4, 5,6, 7,8, 9,10, 11],
        ticktext = ['food', 'animal', 'furniture', 'electronic', 'kitchen', 'vehicle',
       'person', 'outdoor', 'accessory', 'sports', 'appliance', 'indoor']
    )
)

In [31]:
class_names = image_label_df.columns[2:]
class_names.values

array(['food', 'animal', 'furniture', 'electronic', 'kitchen', 'vehicle',
       'person', 'outdoor', 'accessory', 'sports', 'appliance', 'indoor'],
      dtype=object)

### Now we will implement the google net architecture.
Alex Net architecture
--- 
| Layer | Type | Maps | Size | Kernel size | Stride | Padding | Activation |
| ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- |
| Out | Fully connected | – | 1,000 | – | – | – | Softmax |
| F10 | Fully connected | – | 4,096 | – | – | – | ReLU |
| F9 | Fully connected| – | 4,096 | – | – | – ReLU |
| S8 | Max pooling | 256 | 6 × 6 | 3 × 3 | 2 | valid | – |
| C7 | Convolution | 256 | 13 × 13 | 3 × 3 | 1 | same | ReLU |
| C6 | Convolution | 384 | 13 × 13 | 3 × 3 | 1 | same | ReLU |
| C5 | Convolution | 384 | 13 × 13 | 3 × 3 | 1 | same | ReLU |
| S4 | Max pooling | 256 | 13 × 13 | 3 × 3 | 2 | valid | – |
| C3 | Convolution | 256 | 27 × 27 | 5 × 5 | 1 | same |ReLU |
| S2 | Max pooling | 96 | 27 × 27 | 3 × 3 | 2 | valid | – |
| C1 | Convolution | 96 | 55 × 55 | 11 × 11 | 4 |valid | ReLU |
| In | Input | 3 (RGB) | 227 × 227 | – | – | – | – |
---

In [36]:
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        # resizing the image to alex net required size
        self.input_size = (227,227)
        self.transform = Resize(self.input_size)
        # first convolution layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=(11,11), stride=4, padding='valid') # out_shape = (96,55,55)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(3,3),stride = 2)  # out_shape = ()

        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256,stride = 1, kernel_size=(5,5), padding = 'same') # out_shape = ()
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(3,3),stride =2) # out_shape = ()
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=(3,3), stride = 1, padding = 'same') # out_shape = ()
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=(3,3), stride = 1, padding='same') # out_shape = ()
        self.relu4 = nn.ReLU()
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=(3,3), stride = 1, padding='same') # out_shape = ()
        self.relu5 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=(3,3),stride =2) # out_shape = (256,6,6)
        self.fc1 = nn.Linear(256*6*6,4096) # out_shape = (128,4096)
        self.relu6 = nn.ReLU()
        self.fc2 = nn.Linear(4096, 4096) # out_shape = (128,4096)
        self.relu7 = nn.ReLU()
        self.fc3 = nn.Linear(4096, 128) # out_shape = (128,128)
        self.relu8 = nn.ReLU()

        # computation for the multi-label heads
        
        self.head1 = nn.Linear(128,2)
        self.sig1 = nn.Sigmoid()
        self.head2 = nn.Linear(128,2)
        self.sig2 = nn.Sigmoid()
        self.head3 = nn.Linear(128,2)
        self.sig3 = nn.Sigmoid()
        self.head4 = nn.Linear(128,2)
        self.sig4 = nn.Sigmoid()
        self.head5 = nn.Linear(128,2)
        self.sig5 = nn.Sigmoid()
        self.head6 = nn.Linear(128,2)
        self.sig6 = nn.Sigmoid()
        self.head7 = nn.Linear(128,2)
        self.sig7 = nn.Sigmoid()
        self.head8 = nn.Linear(128,2)
        self.sig8 = nn.Sigmoid()
        self.head9 = nn.Linear(128,2)
        self.sig9 = nn.Sigmoid()
        self.head10 = nn.Linear(128,2)
        self.sig10 = nn.Sigmoid()
        self.head11 = nn.Linear(128,2)
        self.sig11 = nn.Sigmoid()
        self.head12 = nn.Linear(128,2)
        self.sig12 = nn.Sigmoid()

    def forward(self, x):
        out = self.transform(x)
        print(out.shape)

        out = self.conv1(out)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = self.conv3(out)
        out = self.relu3(out)
        out = self.conv4(out)
        out = self.relu4(out)
        out = self.conv5(out)
        out = self.relu5(out)
        out = self.maxpool3(out)
        # print(out.shape)
        out = out.view(128, -1)
        out = self.fc1(out)
        out = self.relu6(out)
        out = self.fc2(out)
        out = self.relu7(out)
        out = self.fc3(out)
        out = self.relu8(out)


        # Now we will define the computations of the 12 heads and heads and put all the outputs into one tensor and return that tensor.
        # The out tensor would of the shape (12,32,2).
        self.out_head1 = self.sig1(self.head1(out))  # 'food'
        self.out_head2 = self.sig2(self.head2(out)) # animal
        self.out_head3 = self.sig3(self.head3(out)) # furniture
        self.out_head4 = self.sig4(self.head4(out)) # electronic
        self.out_head5 = self.sig5(self.head5(out)) # kitchen
        self.out_head6 = self.sig6(self.head6(out)) # vehicle
        self.out_head7 = self.sig7(self.head7(out)) # person
        self.out_head8 = self.sig8(self.head8(out)) # outdoor
        self.out_head9 = self.sig9(self.head9(out)) # accessory
        self.out_head10 = self.sig10(self.head10(out)) # sports
        self.out_head11 = self.sig11(self.head11(out)) # appliance
        self.out_head12 = self.sig12(self.head12(out)) # indoor
        out_list = [self.out_head1,self.out_head2,self.out_head3,self.out_head4,self.out_head5,self.out_head6,self.out_head7,self.out_head8,
                    self.out_head9,self.out_head10,self.out_head11,self.out_head12]

        out_tensor = torch.stack(out_list, dim = 1)

        return out_tensor

In [38]:
exp_train_data, exp_train_label = [],[]

for i in range(128):
    img, labels,_ = train_coco[i]
    exp_train_data.append(img)
    exp_train_label.append(labels)
# exp_train_data = torch.tensor(exp_train_data)
# exp_train_label = torch.tensor(exp_train_label)
# print(exp_train_data.shape)
# print(exp_train_label.shape)
# print(len(exp_train_data), len(exp_train_label))
exp_train_data = torch.stack(exp_train_data)
exp_train_label = torch.stack(exp_train_label)
exp_train_data.shape

torch.Size([128, 3, 128, 128])

In [39]:
conv = AlexNet()
o = conv(exp_train_data)
print(o[0])
c = torch.nn.BCELoss()
c(o, exp_train_label)

torch.Size([128, 3, 227, 227])
tensor([[0.5014, 0.4606],
        [0.5052, 0.5172],
        [0.5125, 0.5468],
        [0.4614, 0.4723],
        [0.4669, 0.4502],
        [0.5514, 0.5005],
        [0.5617, 0.5364],
        [0.4698, 0.4598],
        [0.5029, 0.5069],
        [0.4811, 0.5110],
        [0.4826, 0.5524],
        [0.4917, 0.5614]], grad_fn=<SelectBackward0>)


tensor(0.7021, grad_fn=<BinaryCrossEntropyBackward0>)

In [32]:
summary(conv)

Layer (type:depth-idx)                   Param #
AlexNet                                  --
├─Resize: 1-1                            --
├─Conv2d: 1-2                            34,944
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Conv2d: 1-5                            614,656
├─ReLU: 1-6                              --
├─MaxPool2d: 1-7                         --
├─Conv2d: 1-8                            885,120
├─ReLU: 1-9                              --
├─Conv2d: 1-10                           1,327,488
├─ReLU: 1-11                             --
├─Conv2d: 1-12                           884,992
├─ReLU: 1-13                             --
├─MaxPool2d: 1-14                        --
├─Linear: 1-15                           37,752,832
├─ReLU: 1-16                             --
├─Linear: 1-17                           16,781,312
├─ReLU: 1-18                             --
├─Linear: 1-19                           524,416
├─ReLU: 1-20            

We can now see that our alex net is defined clearly. Now we just need to run out batches through the network and work out the training and the validation loop.

In [29]:
# Now we will begin to write the training loop that will ingest the data and output the result.
def training_loop(epochs, conv_model,train_dataloader,device,optimizer= None, val_dataloader=None, loss_fn = None):
    
    for epoch in trange(epochs):
        train_losses = []
        # print("Training is Progressing")
        for img, label,_ in tqdm(train_dataloader, nrows=1):
            batch_losses = []
            img = img.to(device)
            label = label.to(device)

            # Now put the training data into the model to get the logits
            pred = conv_model(img)

            # Now we will zero out the optimizer
            optimizer.zero_grad()
            
            # calculate the loss
            loss = loss_fn(pred, label)

            out_loss = loss.clone().to(device=torch.device('cpu'))

            # backpropagate
            loss.backward()
            # Next we will update the parameters
            optimizer.step()
            # print(out_loss.item())
            train_losses.append(out_loss.item())
        
        # val_loss_list = validation_loop(model = conv_model, val_dataloader=val_dataloader, device = device, loss_fn= loss_fn)
        print(f"Epoch: {epoch} | Train_loss: {torch.tensor(train_losses).mean().item()}")# | Val_loss : {val_loss_list.mean()}")
        # break
            

    return

In [24]:
# train_coco = CocoDataset(image_label_df=image_label_df, val_stride=5)
# val_coco = CocoDataset(image_label_df=image_label_df,is_val_set_bool=True)

# train_dataloader = DataLoader(dataset=train_coco, batch_size=128, pin_memory=True, drop_last=True)
# val_dataloader = DataLoader(dataset=val_coco, batch_size=128,pin_memory=True, drop_last = True)

test_coco = CocoDataset(image_label_df=image_label_df, test_data_set=True, test_stride=50)

test_coco_dataloader = DataLoader(dataset=test_coco, batch_size=128, pin_memory=True, drop_last=True)

In [71]:
print(len(test_coco))

1334


In [25]:
# No we will define the model
alex_conv = AlexNet()
bce_loss = torch.nn.BCELoss()
alex_conv = alex_conv.to(device=device)
optimizer_adam = optim.Adam(alex_conv.parameters(), lr = 0.001)

In [30]:
training_loop(epochs = 10, conv_model=alex_conv, train_dataloader=test_coco_dataloader,
                 val_dataloader=val_dataloader,optimizer=optimizer_adam, loss_fn=bce_loss, device = device)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0 | Train_loss: 23.11849021911621


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Train_loss: 20.970050811767578


...

Epoch: 7 | Train_loss: 20.970050811767578


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 8 | Train_loss: 20.970050811767578


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 9 | Train_loss: 20.970050811767578


#### So there are two types of resenets. 
1. The vanilla resnets uses a resblock of 2 3x3 convolutions and skip connections.
2. The second known as the bottleneck resblock uses 1x1 convolution then a 3x3 conv then a 1x1 conv:
    1. The first 1x1 conv layer reduces the number of filters of the input from 256 to 64.
    2. The second layer uses a 3x3 conv layer with 64 filters to learn the residual mapping.
    3. The third layer uses a 1x1 conv layer with 256 filters to again increase the number of channels to 256 of the output.


#### Now lets create a ResNet and see if it helps with our multilabel classification problem

In [85]:
# First we would implement the resblock which has two 3x3 convolution and see how much we can get out of that training on the coco image dataset.
class ResBlock64(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same')
        self.b1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding = 'same')
        self.b2 =nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        # Now lets define the computation of the resblock
        out = self.relu1(self.b1(self.conv1(x)))
        out = self.b2(self.conv2(out))
        out_final = self.relu2(out + x)
        
        return out_final

class ResBlock128(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding='same')
        self.b1 = nn.BatchNorm2d(128)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding = 'same')
        self.b2 =nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        # Now lets define the computation of the resblock
        out = self.relu1(self.b1(self.conv1(x)))
        out = self.b2(self.conv2(out))
        out_final = self.relu2(out + x)
        return out_final

class ResBlock256(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding='same')
        self.b1 = nn.BatchNorm2d(256)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding = 'same')
        self.b2 =nn.BatchNorm2d(256)
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        # Now lets define the computation of the resblock
        out = self.relu1(self.b1(self.conv1(x)))
        out = self.b2(self.conv2(out))
        out_final = self.relu2(out + x)
        return out_final

class ResBlock512(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding='same')
        self.b1 = nn.BatchNorm2d(512)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding = 'same')
        self.b2 =nn.BatchNorm2d(512)
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        # Now lets define the computation of the resblock
        out = self.relu1(self.b1(self.conv1(x)))
        out = self.b2(self.conv2(out))
        out_final = self.relu2(out + x)
        return out_final

Now we have defined the resblocks. Now we need to define the full resnet

In [86]:
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()

        # The first layer will input an  image of 224x224
        self.input_conv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(7,7), padding = 'same') # in = (3,224,224) | out = (64,224,224)
        self.max_pool1 = nn.MaxPool2d(2)  # in = (64,224,224) | out = (64,128,128)

        self.resblocks64 = nn.Sequential(
            *(3*[ResBlock64()]))                      # in = (64,128,128) | out = (64,128,128)
        self.max_pool2 = nn.MaxPool2d(2)            # in = (64,128,128) | out = (64,64,64)
        self.channel_inc1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), padding='same') # in = (64,64,64) | out = (128,64,64)

        self.resblocks128 = nn.Sequential(
            *(4*[ResBlock128()]))                     # in = (128,64,64) | out = (128,64,64)
        self.max_pool3 = nn.MaxPool2d(2)            # in = (128,64,64) | out = (128,32,32)
        self.channel_inc2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), padding='same')   # in = (128,32,32) | out = (256,32,32)

        self.resblocks256 = nn.Sequential(      
            *(6*[ResBlock256()]))                    # in = (256,32,32) | out = (256,32,32)
        self.max_pool4 = nn.MaxPool2d(2)            # in = (256,16,16) | out = (256,16,16)
        self.channel_inc3 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3), padding='same')   # in = (512,16,16) | out = (512,16,16)

        self.resblocks512 = nn.Sequential(
            *(3*[ResBlock512()]))                     # in = (512,16,16) | out = (512,16,16)
        self.max_pool5 = nn.MaxPool2d(2)            # in = (512,8,8) | out = (512,8,8)

        # input to this layer would be (512,8,8)
        self.fc1 = nn.Linear(512*8*8, 256)
        self.bfc1 = nn.BatchNorm2d(256)
        self.relu1 = nn.ReLU()

        self.fc2 = nn.Linear(256, 128)
        self.bfc2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()

        # self.fc3 = nn.Linear(512, 128)
        # self.bfc3 = nn.BatchNorm2d(128)
        # self.relu3 = nn.ReLU()

        # Now we will need to define our output layers
        # computation for the multi-label heads
        
        self.head1 = nn.Linear(128,2)
        self.sig1 = nn.Sigmoid()
        self.head2 = nn.Linear(128,2)
        self.sig2 = nn.Sigmoid()
        self.head3 = nn.Linear(128,2)
        self.sig3 = nn.Sigmoid()
        self.head4 = nn.Linear(128,2)
        self.sig4 = nn.Sigmoid()
        self.head5 = nn.Linear(128,2)
        self.sig5 = nn.Sigmoid()
        self.head6 = nn.Linear(128,2)
        self.sig6 = nn.Sigmoid()
        self.head7 = nn.Linear(128,2)
        self.sig7 = nn.Sigmoid()
        self.head8 = nn.Linear(128,2)
        self.sig8 = nn.Sigmoid()
        self.head9 = nn.Linear(128,2)
        self.sig9 = nn.Sigmoid()
        self.head10 = nn.Linear(128,2)
        self.sig10 = nn.Sigmoid()
        self.head11 = nn.Linear(128,2)
        self.sig11 = nn.Sigmoid()
        self.head12 = nn.Linear(128,2)
        self.sig12 = nn.Sigmoid()


    def forward(self, X):
        """ This function would define the computations of the layers to produce the output"""
        assert X.shape == (32,3,224,224)
        print(self.input_conv(X).shape)
        # output of the first layer conv and maxpool
        out = self.max_pool1(self.input_conv(X))

        # output of the first resblock, maxpool2, channel_inc1
        out = self.channel_inc1(self.max_pool2(self.resblocks64(out)))

        # output of  resblock128
        out = self.channel_inc2(self.max_pool3(self.resblocks128(out)))

        # output after resblock256
        out = self.channel_inc3(self.max_pool4(self.resblocks256(out)))

        # output after resblock 512
        out = self.max_pool5(self.resblocks512(out))

        # Now we will reshape the tensor to have a
        out = out.view(128,-1)

        # output of the first fully conected layer
        out = self.relu1(self.bfc1(self.fc1(out)))

        # output after second fully connected layer
        out = self.relu2(self.bfc2(self.fc2(out)))

        # output after the third fully connected layer
        # out = self.relu3(self.bfc3(self.fc3(out)))


        # Now we will define the computations of the 12 heads and heads and put all the outputs into one tensor and return that tensor.
        self.out_head1 = self.sig1(self.head1(out))  # 'food'
        self.out_head2 = self.sig2(self.head2(out)) # animal
        self.out_head3 = self.sig3(self.head3(out)) # furniture
        self.out_head4 = self.sig4(self.head4(out)) # electronic
        self.out_head5 = self.sig5(self.head5(out)) # kitchen
        self.out_head6 = self.sig6(self.head6(out)) # vehicle
        self.out_head7 = self.sig7(self.head7(out)) # person
        self.out_head8 = self.sig8(self.head8(out)) # outdoor
        self.out_head9 = self.sig9(self.head9(out)) # accessory
        self.out_head10 = self.sig10(self.head10(out)) # sports
        self.out_head11 = self.sig11(self.head11(out)) # appliance
        self.out_head12 = self.sig12(self.head12(out)) # indoor
        out_list = [self.out_head1,self.out_head2,self.out_head3,self.out_head4,self.out_head5,self.out_head6,self.out_head7,self.out_head8,
                    self.out_head9,self.out_head10,self.out_head11,self.out_head12]

        out_tensor = torch.stack(out_list, dim = 1)

In [87]:

res_conv = ResNet()
res_conv = res_conv.to(device=device)
summary(res_conv)

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 4.00 GiB total capacity; 2.87 GiB already allocated; 0 bytes free; 2.90 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [16]:
# Now lets get a batch of data and try to our model into our data.
exp_train_data, exp_train_label = [],[]

for i in range(32):
    img, labels,_ = train_coco[i]
    exp_train_data.append(img)
    exp_train_label.append(labels)
# exp_train_data = torch.tensor(exp_train_data)
# exp_train_label = torch.tensor(exp_train_label)
# print(exp_train_data.shape)
# print(exp_train_label.shape)
# print(len(exp_train_data), len(exp_train_label))
exp_train_data = torch.stack(exp_train_data)
exp_train_label = torch.stack(exp_train_label)
exp_train_data.shape

torch.Size([32, 3, 224, 224])

In [17]:
exp_train_data = exp_train_data.to(device=device)
exp_train_label = exp_train_label.to(device=device)

In [18]:
res_conv(exp_train_data)

torch.Size([32, 64, 224, 224])


OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB (GPU 0; 4.00 GiB total capacity; 2.79 GiB already allocated; 0 bytes free; 2.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

The above model that we defined is too large to fit into memory. So we will define a the model with two 1x1 convolutions and one 3x3 convolutions.

In [13]:
# First we would implement the resblock which has two 3x3 convolution and see how much we can get out of that training on the coco image dataset.
class ResBlock64(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size = 1, padding = 'same')
        self.b1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding='same')
        self.b2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=1, padding = 'same')
        self.b3 =nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU()
    
    def forward(self, x):
        # Now lets define the computation of the resblock
        out = self.relu1(self.b1(self.conv1(x)))
        out = self.relu2(self.b2(self.conv2(out)))
        out = self.b3(self.conv3(out))
        out_final = self.relu3(out + x)
        
        return out_final

class ResBlock128(nn.Module):
    def __init__(self):
        super().__init__()
        # First 1x1 conv layer
        self.conv1 = nn.Conv2d(in_channels=128, out_channels=32, kernel_size=1, padding='same')
        self.b1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        # Second 3x3 conv layer
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding = 'same')
        self.b2 =nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()

        # Third 1x1 conv layer
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=128, kernel_size=1, padding = 'same')
        self.b3 =nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()

    
    def forward(self, x):
        # Now lets define the computation of the resblock
        out = self.relu1(self.b1(self.conv1(x)))
        out = self.relu2(self.b2(self.conv2(out)))
        out = self.b3(self.conv3(out))
        out_final = self.relu3(out + x)
        return out_final

class downsampleBlock(nn.Module):
    """ This class defines a downsampling block which is used to half the image size and double the channels of an image."""
    def __init__(self, in_chan, out_chan):
        super().__init__()
        # Define the first 1x1 conv layer. This layer would increase the channels of the input
        self.conv1 = nn.Conv2d(in_channels = in_chan, out_channels=out_chan, kernel_size=1, stride = 2)
        self.b1 = nn.BatchNorm2d(out_chan)
        self.relu1 = nn.ReLU()

        # Then define the 3x3 conv layer. This layer would decrease the size of the image while making the channels constant.
        self.conv2 = nn.Conv2d(in_channels = out_chan, out_channels=out_chan, kernel_size=3, padding='same')
        self.b2 = nn.BatchNorm2d(out_chan)
        self.relu2 = nn.ReLU()

        # Now define the third 1x1 layer. This layer would increase the channels of the input so that it 
        self.conv3 = nn.Conv2d(in_channels = in_chan, out_channels=out_chan, kernel_size = 1, stride = 2)
        self.b3 = nn.BatchNorm2d(out_chan)


    def forward(self, X):
        out = self.relu1(self.b1(self.conv1(X)))
        out = self.b2(self.conv2(out))

        out_short = self.b3(self.conv3(X))

        out_final = self.relu2(out + out_short)

        return out_final

class ResBlock256(nn.Module):
    def __init__(self):
        super().__init__()
        # First 1x1 conv layer
        self.conv1 = nn.Conv2d(in_channels=256, out_channels=64, kernel_size=1, padding='same')
        self.b1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()
        # Second 3x3 conv layer
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding = 'same')
        self.b2 =nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()

        # Third 1x1 conv layer
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=1, padding = 'same')
        self.b3 =nn.BatchNorm2d(256)
        self.relu3 = nn.ReLU()

    
    def forward(self, x):
        # Now lets define the computation of the resblock
        out = self.relu1(self.b1(self.conv1(x)))
        out = self.relu2(self.b2(self.conv2(out)))
        out = self.b3(self.conv3(out))
        out_final = self.relu3(out + x)
        return out_final

class ResBlock512(nn.Module):
    def __init__(self):
        super().__init__()
        # First 1x1 conv layer
        self.conv1 = nn.Conv2d(in_channels=512, out_channels=128, kernel_size=1, padding='same')
        self.b1 = nn.BatchNorm2d(128)
        self.relu1 = nn.ReLU()
        # Second 3x3 conv layer
        self.conv2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding = 'same')
        self.b2 =nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()

        # Third 1x1 conv layer
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=512, kernel_size=1, padding = 'same')
        self.b3 =nn.BatchNorm2d(512)
        self.relu3 = nn.ReLU()

    
    def forward(self, x):
        # Now lets define the computation of the resblock
        out = self.relu1(self.b1(self.conv1(x)))
        out = self.relu2(self.b2(self.conv2(out)))
        out = self.b3(self.conv3(out))
        out_final = self.relu3(out + x)
        return out_final

In [65]:
p = torch.manual_seed(42)
x = torch.randn(32,64,28,28)
r_b64 = downsampleBlock(64,128)

y = r_b64(x)
print(y.shape)


torch.Size([32, 128, 14, 14])


#### Now lets make the full architecture.

In [14]:
class ResNet(nn.Module):
    def __init__(self,batch_size = None):
        super().__init__()
        self.batch_size = batch_size
        # The first layer will input an  image of 224x224
        self.input_conv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(7,7), padding = 'same') # in = (3,224,224) | out = (64,224,224)
        self.max_pool1 = nn.MaxPool2d(2)  # in = (64,224,224) | out = (64,112,112)

        self.resblocks64 = nn.Sequential(
            *(3*[ResBlock64()]+[downsampleBlock(64,128)]))                      # in = (64,128,128) | out = (64,128,128)
        # self.max_pool2 = nn.MaxPool2d(2)            # in = (64,128,128) | out = (64,64,64)
        # self.channel_inc1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), padding='same') # in = (64,64,64) | out = (128,64,64)

        self.resblocks128 = nn.Sequential(
            *(3*[ResBlock128()]+[downsampleBlock(128,256)]))                     # in = (128,64,64) | out = (128,64,64)
        # self.max_pool3 = nn.MaxPool2d(2)            # in = (128,64,64) | out = (128,32,32)
        # self.channel_inc2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), padding='same')   # in = (128,32,32) | out = (256,32,32)

        self.resblocks256 = nn.Sequential(      
            *(5*[ResBlock256()]+[downsampleBlock(256,512)]))                    # in = (256,32,32) | out = (256,32,32)
        # self.max_pool4 = nn.MaxPool2d(2)            # in = (256,16,16) | out = (256,16,16)
        # self.channel_inc3 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3), padding='same')   # in = (512,16,16) | out = (512,16,16)

        self.resblocks512 = nn.Sequential(
            *(2*[ResBlock512()]))                     # in = (512,16,16) | out = (512,16,16)
        # self.max_pool5 = nn.MaxPool2d(2)            # in = (512,8,8) | out = (512,8,8)

        # Now we would use a Global Average Pooling Layer which is just a mean on the first two image dimensions excluding the channel dim.
        # But that would just be an operation in the forward pass

        self.fc2 = nn.Linear(512, 128)
        self.bfc2 = nn.BatchNorm1d(128)
        self.relu_fc2 = nn.ReLU()

        # self.fc3 = nn.Linear(512, 128)
        # self.bfc3 = nn.BatchNorm2d(128)
        # self.relu3 = nn.ReLU()

        # Now we will need to define our output layers
        # computation for the multi-label heads
        
        self.head1 = nn.Linear(128,2)
        self.sig1 = nn.Sigmoid()
        self.head2 = nn.Linear(128,2)
        self.sig2 = nn.Sigmoid()
        self.head3 = nn.Linear(128,2)
        self.sig3 = nn.Sigmoid()
        self.head4 = nn.Linear(128,2)
        self.sig4 = nn.Sigmoid()
        self.head5 = nn.Linear(128,2)
        self.sig5 = nn.Sigmoid()
        self.head6 = nn.Linear(128,2)
        self.sig6 = nn.Sigmoid()
        self.head7 = nn.Linear(128,2)
        self.sig7 = nn.Sigmoid()
        self.head8 = nn.Linear(128,2)
        self.sig8 = nn.Sigmoid()
        self.head9 = nn.Linear(128,2)
        self.sig9 = nn.Sigmoid()
        self.head10 = nn.Linear(128,2)
        self.sig10 = nn.Sigmoid()
        self.head11 = nn.Linear(128,2)
        self.sig11 = nn.Sigmoid()
        self.head12 = nn.Linear(128,2)
        self.sig12 = nn.Sigmoid()



    def forward(self, X):
        """ This function would define the computations of the layers to produce the output"""
        # assert X.shape == (2,3,224,224)
        print(self.input_conv(X).shape)
        # output of the first layer conv and maxpool
        out = self.max_pool1(self.input_conv(X))

        # output of the first resblock, maxpool2, channel_inc1
        out = self.resblocks64(out)

        # output of  resblock128
        out = self.resblocks128(out)

        # output after resblock256
        out = self.resblocks256(out)

        # output after resblock 512
        out =  self.resblocks512(out)

        # Then we would use a global average pooling layer i.e mean on the image_height and width using torch.mean()
        out = torch.mean(out, dim = (2,3), keepdim=True)
        # Now we will reshape the tensor to have a
        # out = out.view(128,-1)

        out = out.view(self.batch_size,-1)
        # output of the first fully conected layer
        # out = self.relu1(self.bfc1(self.fc1(out)))
        print(f"The output after the average pooling op is {out.shape}")

        out = self.fc2(out)
        out = self.bfc2(out)
        out = self.relu_fc2(out)
        # output after second fully connected layer
        # out = self.relu_fc2(self.bfc2(self.fc2(out)))

        # output after the third fully connected layer
        # out = self.relu3(self.bfc3(self.fc3(out)))


        # Now we will define the computations of the 12 heads and heads and put all the outputs into one tensor and return that tensor.
        self.out_head1 = self.sig1(self.head1(out))  # 'food'
        self.out_head2 = self.sig2(self.head2(out)) # animal
        self.out_head3 = self.sig3(self.head3(out)) # furniture
        self.out_head4 = self.sig4(self.head4(out)) # electronic
        self.out_head5 = self.sig5(self.head5(out)) # kitchen
        self.out_head6 = self.sig6(self.head6(out)) # vehicle
        self.out_head7 = self.sig7(self.head7(out)) # person
        self.out_head8 = self.sig8(self.head8(out)) # outdoor
        self.out_head9 = self.sig9(self.head9(out)) # accessory
        self.out_head10 = self.sig10(self.head10(out)) # sports
        self.out_head11 = self.sig11(self.head11(out)) # appliance
        self.out_head12 = self.sig12(self.head12(out)) # indoor
        out_list = [self.out_head1,self.out_head2,self.out_head3,self.out_head4,self.out_head5,self.out_head6,self.out_head7,self.out_head8,
                    self.out_head9,self.out_head10,self.out_head11,self.out_head12]

        out_tensor = torch.stack(out_list, dim = 1)
        print(out_tensor.shape)

In [15]:
res_c = ResNet(batch_size = 32)
summary(res_c, input_size = (32,3,224,224))

torch.Size([32, 64, 224, 224])
The output after the average pooling op is torch.Size([32, 512])
torch.Size([32, 12, 2])


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   --                        --
├─Conv2d: 1-1                            [32, 64, 224, 224]        9,472
├─Conv2d: 1-2                            [32, 64, 224, 224]        (recursive)
├─MaxPool2d: 1-3                         [32, 64, 112, 112]        --
├─Sequential: 1-4                        [32, 128, 56, 56]         --
│    └─ResBlock64: 2-1                   [32, 64, 112, 112]        --
│    │    └─Conv2d: 3-1                  [32, 32, 112, 112]        2,080
│    │    └─BatchNorm2d: 3-2             [32, 32, 112, 112]        64
│    │    └─ReLU: 3-3                    [32, 32, 112, 112]        --
│    │    └─Conv2d: 3-4                  [32, 32, 112, 112]        9,248
│    │    └─BatchNorm2d: 3-5             [32, 32, 112, 112]        64
│    │    └─ReLU: 3-6                    [32, 32, 112, 112]        --
│    │    └─Conv2d: 3-7                  [32, 64, 112, 112]        

In [16]:
# Now lets get a batch of data and try to our model into our data.
exp_train_data, exp_train_label = [],[]

for i in range(32):
    img, labels,_ = train_coco[i]
    exp_train_data.append(img)
    exp_train_label.append(labels)
# exp_train_data = torch.tensor(exp_train_data)
# exp_train_label = torch.tensor(exp_train_label)
# print(exp_train_data.shape)
# print(exp_train_label.shape)
# print(len(exp_train_data), len(exp_train_label))
exp_train_data = torch.stack(exp_train_data)
exp_train_label = torch.stack(exp_train_label)
exp_train_data.shape

torch.Size([32, 3, 224, 224])

In [17]:
exp_train_data = exp_train_data.to(device=device)
exp_train_label = exp_train_label.to(device=device)

In [18]:
res_c = res_c.to(device=device)
res_c(exp_train_data)

torch.Size([32, 64, 224, 224])


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 4.00 GiB total capacity; 2.82 GiB already allocated; 0 bytes free; 2.87 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF